# CTU-Net

<img src='images/ctunet.png' width='600'/>

## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, backend, Model
from tensorflow.keras.applications import EfficientNetB4, EfficientNetV2S, ResNet50V2, DenseNet201

## Block functions

<img src='images/dbb.png' width='600'/>

In [2]:
def DB_block(input1, num_filters):
    x = layers.BatchNormalization()(input1)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    c1 = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    
    x = layers.concatenate([input1,c1]) 
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    
    x = layers.concatenate([input1,x,c1])
    return x

def DBB_block(input1, input2, num_filters):
    x = DB_block(input1, num_filters)
    
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(x)
    x = layers.Activation("relu")(x)
    x = layers.BatchNormalization()(x)
    
    # Rescale input2 to have the same filter dimension as x #
    input2 = layers.Conv2D(num_filters, (1, 1), padding="same")(input2)
    ###################################################
    x = layers.Add()([x, input2])
    
    x = layers.Conv2D(num_filters, (1, 1), strides=(2, 2), padding="same")(x)
    # Reduce size of x to match its original size before Conv2DTranspose#
    #x = layers.MaxPooling2D((2,2))(x)
    ###################################################
    return x

<img src='images/scab.png' width='600'/>

In [3]:
def SCAB_block(input1, input2, num_filters, final=False):
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(input1)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.Activation("sigmoid")(x)
    
    if final:
        input2 = layers.Conv2D(num_filters, (1, 1), padding="same")(input2)
        
    x = layers.Multiply()([x, input2])
    
    x = layers.concatenate([input1,x])
    
    c1 = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.GlobalAveragePooling2D(keepdims=True)(c1)
    
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.Activation("sigmoid")(x)
    
    x = layers.Multiply()([x, c1])
    
    x = layers.Add()([x, c1])
    return x

In [4]:
def conv_block(input, num_filters):
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def deconv_block(input, num_filters):
    x = layers.Conv2DTranspose(num_filters, 1, padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def decoder_block(input, skip_output, num_filters, final=False):
    x = conv_block(input, num_filters)
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = SCAB_block(x, skip_output, num_filters, final)
    x = DB_block(x, num_filters)
    return x

def bottleneck(input):
    x = layers.MaxPooling2D((2,2))(input)
    #x = layers.GlobalAveragePooling2D()(x)
    return x

## EfficientNetB4

In [4]:
def EfficientNetB4_CTUnet(input_shape=(512, 512, 3)):
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    EffNetB4 = EfficientNetB4(include_top=False, weights="imagenet", input_tensor=inputs)

    # Encoder
    res0 = EffNetB4.get_layer('rescaling_1').output  # 512 x 512
    res1 = EffNetB4.get_layer('block2a_expand_activation').output  # 256 x 256
    res2 = EffNetB4.get_layer('block3a_expand_activation').output  # 128 x 128
    res3 = EffNetB4.get_layer('block4a_expand_activation').output  # 64 x 64
    res4 = EffNetB4.get_layer('block6a_expand_activation').output  # 32 x 32
    
    # Skip connection blocks
    skip0 = DB_block(res0, 32)
    skip1 = DBB_block(res1, skip0, 64)
    skip2 = DBB_block(res2, skip1, 128)
    skip3 = DBB_block(res3, skip2, 256)
    skip4 = DBB_block(res4, skip3, 512)
    
    # Bottleneck
    b1 = bottleneck(res4)  # 16 x 16

    # Decoder
    d1 = decoder_block(b1, skip4, 512)  # 32 x 32
    d2 = decoder_block(d1, skip3, 256)    # 64 x 64
    d3 = decoder_block(d2, skip2, 128)   # 128 x 128
    d4 = decoder_block(d3, skip1, 64)   # 256 x 256
    d5 = decoder_block(d4, skip0, 32, True)   # 512 x 512

    # Output
    outputs = conv_block(d5, 16)
    outputs = deconv_block(outputs, 16)
    outputs = conv_block(outputs, 16)
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(outputs)

    model = Model(inputs, outputs, name="EfficientNetB4_CTU-Net")
    return model


In [5]:
model = EfficientNetB4_CTUnet()

In [6]:
model.summary()

Model: "EfficientNetB4_CTU-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['normalizat

 block2a_expand_conv (Conv2D)   (None, 256, 256, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 256, 256, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 256, 256, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 257, 257, 14  0          ['block2a_expand_activation[0][0]
 g2D)     

                                                                                                  
 block2c_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block2c_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block2c_expand_activation (Act  (None, 128, 128, 19  0          ['block2c_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 128, 128, 19  1728       ['block2c_expand_activation[0][0]
 D)                             2)                               ']                               
                                                                                                  
 block2c_b

                                2)                                                                
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block3a_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block3a_expand_activation (Act  (None, 128, 128, 19  0          ['block3a_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block3a_dwconv_pad (ZeroPaddin  (None, 131, 131, 19  0          ['block3a_expand_activation[0][0]
 g2D)                           2)                               ']                               
          

                                                                                                  
 block3c_dwconv (DepthwiseConv2  (None, 64, 64, 336)  8400       ['block3c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3c_bn (BatchNormalization  (None, 64, 64, 336)  1344       ['block3c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3c_activation (Activation  (None, 64, 64, 336)  0          ['block3c_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3c_s

...

 batch_normalization_35 (BatchN  (None, 64, 64, 256)  1024       ['conv2d_50[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_47 (Activation)     (None, 64, 64, 256)  0           ['batch_normalization_35[0][0]'] 
                                                                                                  
 conv2d_51 (Conv2D)             (None, 64, 64, 256)  590080      ['activation_47[0][0]']          
                                                                                                  
 concatenate_15 (Concatenate)   (None, 64, 64, 768)  0           ['add_5[0][0]',                  
                                                                  'conv2d_51[0][0]',              
                                                                  'conv2d_49[0][0]']              
          

                                                                                                  
 concatenate_17 (Concatenate)   (None, 128, 128, 25  0           ['add_6[0][0]',                  
                                6)                                'conv2d_59[0][0]']              
                                                                                                  
 batch_normalization_40 (BatchN  (None, 128, 128, 25  1024       ['concatenate_17[0][0]']         
 ormalization)                  6)                                                                
                                                                                                  
 activation_56 (Activation)     (None, 128, 128, 25  0           ['batch_normalization_40[0][0]'] 
                                6)                                                                
                                                                                                  
 conv2d_60

 batch_normalization_44 (BatchN  (None, 256, 256, 64  256        ['add_7[0][0]']                  
 ormalization)                  )                                                                 
                                                                                                  
 activation_64 (Activation)     (None, 256, 256, 64  0           ['batch_normalization_44[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_68 (Conv2D)             (None, 256, 256, 64  4160        ['activation_64[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_45 (BatchN  (None, 256, 256, 64  256        ['conv2d_68[0][0]']              
 ormalizat

                                )                                                                 
                                                                                                  
 global_average_pooling2d_4 (Gl  (None, 1, 1, 32)    0           ['conv2d_76[0][0]']              
 obalAveragePooling2D)                                                                            
                                                                                                  
 conv2d_77 (Conv2D)             (None, 1, 1, 32)     1056        ['global_average_pooling2d_4[0][0
                                                                 ]']                              
                                                                                                  
 activation_72 (Activation)     (None, 1, 1, 32)     0           ['conv2d_77[0][0]']              
                                                                                                  
 conv2d_78

                                                                                                  
 batch_normalization_56 (BatchN  (None, 512, 512, 16  64         ['conv2d_84[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_80 (Activation)     (None, 512, 512, 16  0           ['batch_normalization_56[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_85 (Conv2D)             (None, 512, 512, 1)  17          ['activation_80[0][0]']          
                                                                                                  
Total params: 34,966,880
Trainable params: 34,887,901
Non-trainable params: 78,979
__________________________

## EfficientNetV2S

In [6]:
def EfficientNetV2S_CTUnet(input_shape=(512, 512, 3)):
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    EffNetV2S = EfficientNetV2S(include_top=False, weights="imagenet", input_tensor=inputs)
    
    # Encoder
    res0 = EffNetV2S.get_layer('rescaling').output  # 512 x 512
    res1 = EffNetV2S.get_layer('block1b_add').output  # 256 x 256
    res2 = EffNetV2S.get_layer('block2d_add').output  # 128 x 128
    res3 = EffNetV2S.get_layer('block4a_expand_activation').output  # 64 x 64
    res4 = EffNetV2S.get_layer('block6a_expand_activation').output  # 32 x 32
    
    # Skip connection blocks
    skip0 = DB_block(res0, 32)
    skip1 = DBB_block(res1, skip0, 64)
    skip2 = DBB_block(res2, skip1, 128)
    skip3 = DBB_block(res3, skip2, 256)
    skip4 = DBB_block(res4, skip3, 512)
    
    # Bottleneck
    b1 = bottleneck(res4)  # 16 x 16

    # Decoder
    d1 = decoder_block(b1, skip4, 512)  # 32 x 32
    d2 = decoder_block(d1, skip3, 256)    # 64 x 64
    d3 = decoder_block(d2, skip2, 128)   # 128 x 128
    d4 = decoder_block(d3, skip1, 64)   # 256 x 256
    d5 = decoder_block(d4, skip0, 32, True)   # 512 x 512

    # Output
    outputs = conv_block(d5, 16)
    outputs = deconv_block(outputs, 16)
    outputs = conv_block(outputs, 16)
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(outputs)

    model = Model(inputs, outputs, name="EfficientNetV2S_CTU-Net")
    
    return model

In [7]:
model1 = EfficientNetV2S_CTUnet()

Model: "efficientnetv2-s"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 256, 256, 24  648         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                   

                                                                                                  
 block2c_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block2c_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block2c_expand_activation (Act  (None, 128, 128, 19  0          ['block2c_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block2c_project_conv (Conv2D)  (None, 128, 128, 48  9216        ['block2c_expand_activation[0][0]
                                )                                ']                               
                                                                                                  
 block2c_p

                                                                                                  
 block3c_project_bn (BatchNorma  (None, 64, 64, 64)  256         ['block3c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block3c_drop (Dropout)         (None, 64, 64, 64)   0           ['block3c_project_bn[0][0]']     
                                                                                                  
 block3c_add (Add)              (None, 64, 64, 64)   0           ['block3c_drop[0][0]',           
                                                                  'block3b_add[0][0]']            
                                                                                                  
 block3d_expand_conv (Conv2D)   (None, 64, 64, 256)  147456      ['block3c_add[0][0]']            
          

                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 32)     16416       ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_se_expand (Conv2D)     (None, 1, 1, 512)    16896       ['block4b_se_reduce[0][0]']      
                                                                                                  
 block4b_se_excite (Multiply)   (None, 32, 32, 512)  0           ['block4b_activation[0][0]',     
                                                                  'block4b_se_expand[0][0]']      
                                                                                                  
 block4b_project_conv (Conv2D)  (None, 32, 32, 128)  65536       ['block4b_se_excite[0][0]']      
                                                                                                  
 block4b_p

 block4d_se_excite (Multiply)   (None, 32, 32, 512)  0           ['block4d_activation[0][0]',     
                                                                  'block4d_se_expand[0][0]']      
                                                                                                  
 block4d_project_conv (Conv2D)  (None, 32, 32, 128)  65536       ['block4d_se_excite[0][0]']      
                                                                                                  
 block4d_project_bn (BatchNorma  (None, 32, 32, 128)  512        ['block4d_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block4d_drop (Dropout)         (None, 32, 32, 128)  0           ['block4d_project_bn[0][0]']     
                                                                                                  
 block4d_a

...

                                                                                                  
 block6h_add (Add)              (None, 16, 16, 256)  0           ['block6h_drop[0][0]',           
                                                                  'block6g_add[0][0]']            
                                                                                                  
 block6i_expand_conv (Conv2D)   (None, 16, 16, 1536  393216      ['block6h_add[0][0]']            
                                )                                                                 
                                                                                                  
 block6i_expand_bn (BatchNormal  (None, 16, 16, 1536  6144       ['block6i_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6i_e

                                                                                                  
 block6k_expand_conv (Conv2D)   (None, 16, 16, 1536  393216      ['block6j_add[0][0]']            
                                )                                                                 
                                                                                                  
 block6k_expand_bn (BatchNormal  (None, 16, 16, 1536  6144       ['block6k_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6k_expand_activation (Act  (None, 16, 16, 1536  0          ['block6k_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block6k_d

                                                                                                  
 block6m_expand_bn (BatchNormal  (None, 16, 16, 1536  6144       ['block6m_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6m_expand_activation (Act  (None, 16, 16, 1536  0          ['block6m_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block6m_dwconv2 (DepthwiseConv  (None, 16, 16, 1536  13824      ['block6m_expand_activation[0][0]
 2D)                            )                                ']                               
                                                                                                  
 block6m_b

                                                                                                  
 block6o_expand_activation (Act  (None, 16, 16, 1536  0          ['block6o_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block6o_dwconv2 (DepthwiseConv  (None, 16, 16, 1536  13824      ['block6o_expand_activation[0][0]
 2D)                            )                                ']                               
                                                                                                  
 block6o_bn (BatchNormalization  (None, 16, 16, 1536  6144       ['block6o_dwconv2[0][0]']        
 )                              )                                                                 
                                                                                                  
 block6o_a

ValueError: No such layer: rescaling. Existing layers are: ['input_2', 'rescaling_1', 'stem_conv', 'stem_bn', 'stem_activation', 'block1a_project_conv', 'block1a_project_bn', 'block1a_project_activation', 'block1a_add', 'block1b_project_conv', 'block1b_project_bn', 'block1b_project_activation', 'block1b_drop', 'block1b_add', 'block2a_expand_conv', 'block2a_expand_bn', 'block2a_expand_activation', 'block2a_project_conv', 'block2a_project_bn', 'block2b_expand_conv', 'block2b_expand_bn', 'block2b_expand_activation', 'block2b_project_conv', 'block2b_project_bn', 'block2b_drop', 'block2b_add', 'block2c_expand_conv', 'block2c_expand_bn', 'block2c_expand_activation', 'block2c_project_conv', 'block2c_project_bn', 'block2c_drop', 'block2c_add', 'block2d_expand_conv', 'block2d_expand_bn', 'block2d_expand_activation', 'block2d_project_conv', 'block2d_project_bn', 'block2d_drop', 'block2d_add', 'block3a_expand_conv', 'block3a_expand_bn', 'block3a_expand_activation', 'block3a_project_conv', 'block3

In [ ]:
model1.summary()

## ResNet50V2

In [4]:
def ResNet50V2_CTUnet(input_shape=(512, 512, 3)):
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    ResNet50 = ResNet50V2(include_top=False, weights="imagenet", input_tensor=inputs)
    
    # Encoder
    res0 = ResNet50.get_layer('input_1').output  # 512 x 512
    res1 = ResNet50.get_layer('conv1_conv').output  # 256 x 256
    res2 = ResNet50.get_layer('conv2_block3_1_relu').output  # 128 x 128
    res3 = ResNet50.get_layer('conv3_block4_1_relu').output  # 64 x 64
    res4 = ResNet50.get_layer('conv4_block6_1_relu').output  # 32 x 32

    # Skip connection blocks
    skip0 = DB_block(res0, 32)
    skip1 = DBB_block(res1, skip0, 64)
    skip2 = DBB_block(res2, skip1, 128)
    skip3 = DBB_block(res3, skip2, 256)
    skip4 = DBB_block(res4, skip3, 512)
    
    # Bottleneck
    b1 = bottleneck(res4)  # 16 x 16

    # Decoder
    d1 = decoder_block(b1, skip4, 512)  # 32 x 32
    d2 = decoder_block(d1, skip3, 256)    # 64 x 64
    d3 = decoder_block(d2, skip2, 128)   # 128 x 128
    d4 = decoder_block(d3, skip1, 64)   # 256 x 256
    d5 = decoder_block(d4, skip0, 32, True)   # 512 x 512

    # Output
    outputs = conv_block(d5, 16)
    outputs = deconv_block(outputs, 16)
    outputs = conv_block(outputs, 16)
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(outputs)

    model = Model(inputs, outputs, name="ResNet50V2_CTU-Net")
    return model

In [5]:
model2 = ResNet50V2_CTUnet()

In [6]:
model2.summary()

Model: "ResNet50V2_CTU-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                 

 conv2_block2_out (Add)         (None, 128, 128, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, 128, 128, 25  1024       ['conv2_block2_out[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 128, 128, 25  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 128, 128, 64  16384       ['conv2_block3_preact_relu[0][0]'
          

 n)                                                                                               
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, 66, 66, 128)  0          ['conv3_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 64, 64, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 64, 64, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_blo

 ormalization)                                                                                    
                                                                                                  
 conv4_block1_preact_relu (Acti  (None, 32, 32, 512)  0          ['conv4_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 32, 32, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 32, 32, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block3_2_conv (Conv2D)   (None, 32, 32, 256)  589824      ['conv4_block3_2_pad[0][0]']     
                                                                                                  
 conv4_block3_2_bn (BatchNormal  (None, 32, 32, 256)  1024       ['conv4_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 32, 32, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 32, 32, 1024  263168      ['conv4_block3_2_relu[0][0]']    
          

...

                                                                                                  
 activation_41 (Activation)     (None, 64, 64, 256)  0           ['conv2d_44[0][0]']              
                                                                                                  
 multiply_2 (Multiply)          (None, 64, 64, 256)  0           ['activation_41[0][0]',          
                                                                  'max_pooling2d_5[0][0]']        
                                                                                                  
 concatenate_13 (Concatenate)   (None, 64, 64, 512)  0           ['activation_39[0][0]',          
                                                                  'multiply_2[0][0]']             
                                                                                                  
 conv2d_45 (Conv2D)             (None, 64, 64, 256)  131328      ['concatenate_13[0][0]']         
          

 activation_50 (Activation)     (None, 128, 128, 12  0           ['conv2d_53[0][0]']              
                                8)                                                                
                                                                                                  
 conv2d_54 (Conv2D)             (None, 128, 128, 12  16512       ['activation_50[0][0]']          
                                8)                                                                
                                                                                                  
 activation_51 (Activation)     (None, 128, 128, 12  0           ['conv2d_54[0][0]']              
                                8)                                                                
                                                                                                  
 multiply_4 (Multiply)          (None, 128, 128, 12  0           ['activation_51[0][0]',          
          

 ormalization)                  )                                                                 
                                                                                                  
 activation_58 (Activation)     (None, 128, 128, 64  0           ['batch_normalization_42[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_transpose_7 (Conv2DTran  (None, 256, 256, 64  16448      ['activation_58[0][0]']          
 spose)                         )                                                                 
                                                                                                  
 batch_normalization_43 (BatchN  (None, 256, 256, 64  256        ['conv2d_transpose_7[0][0]']     
 ormalization)                  )                                                                 
          

 batch_normalization_47 (BatchN  (None, 256, 256, 64  256        ['conv2d_70[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_67 (Activation)     (None, 256, 256, 64  0           ['batch_normalization_47[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_71 (Conv2D)             (None, 256, 256, 64  36928       ['activation_67[0][0]']          
                                )                                                                 
                                                                                                  
 concatenate_21 (Concatenate)   (None, 256, 256, 19  0           ['add_7[0][0]',                  
          

 ormalization)                  )                                                                 
                                                                                                  
 activation_75 (Activation)     (None, 512, 512, 32  0           ['batch_normalization_51[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_80 (Conv2D)             (None, 512, 512, 32  9248        ['activation_75[0][0]']          
                                )                                                                 
                                                                                                  
 concatenate_23 (Concatenate)   (None, 512, 512, 64  0           ['add_8[0][0]',                  
                                )                                 'conv2d_80[0][0]']              
          

## DenseNet201

In [4]:
def DenseNet201_CTUnet(input_shape=(512, 512, 3)):
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    DenseNet = DenseNet201(include_top=False, weights="imagenet", input_tensor=inputs)
    
    # Encoder
    res0 = DenseNet.get_layer('input_1').output  # 512 x 512
    res1 = DenseNet.get_layer('conv1/relu').output  # 256 x 256
    res2 = DenseNet.get_layer('pool2_conv').output  # 128 x 128
    res3 = DenseNet.get_layer('pool3_conv').output  # 64 x 64
    res4 = DenseNet.get_layer('pool4_conv').output  # 32 x 32
    
    # Skip connection blocks
    skip0 = DB_block(res0, 32)
    skip1 = DBB_block(res1, skip0, 64)
    skip2 = DBB_block(res2, skip1, 128)
    skip3 = DBB_block(res3, skip2, 256)
    skip4 = DBB_block(res4, skip3, 512)
    
    # Bottleneck
    b1 = bottleneck(res4)  # 16 x 16

    # Decoder
    d1 = decoder_block(b1, skip4, 512)  # 32 x 32
    d2 = decoder_block(d1, skip3, 256)    # 64 x 64
    d3 = decoder_block(d2, skip2, 128)   # 128 x 128
    d4 = decoder_block(d3, skip1, 64)   # 256 x 256
    d5 = decoder_block(d4, skip0, 32, True)   # 512 x 512

    # Output
    outputs = conv_block(d5, 16)
    outputs = deconv_block(outputs, 16)
    outputs = conv_block(outputs, 16)
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(outputs)

    model = Model(inputs, outputs, name="DenseNet201_CTU-Net")
    return model

In [5]:
model3 = DenseNet201_CTUnet()

In [6]:
model3.summary()

Model: "DenseNet201_CTU-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 256, 256, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                

 conv2_block3_concat (Concatena  (None, 128, 128, 16  0          ['conv2_block2_concat[0][0]',    
 te)                            0)                                'conv2_block3_2_conv[0][0]']    
                                                                                                  
 conv2_block4_0_bn (BatchNormal  (None, 128, 128, 16  640        ['conv2_block3_concat[0][0]']    
 ization)                       0)                                                                
                                                                                                  
 conv2_block4_0_relu (Activatio  (None, 128, 128, 16  0          ['conv2_block4_0_bn[0][0]']      
 n)                             0)                                                                
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 128, 128, 12  20480       ['conv2_block4_0_relu[0][0]']    
          

 conv3_block1_1_conv (Conv2D)   (None, 64, 64, 128)  16384       ['conv3_block1_0_relu[0][0]']    
                                                                                                  
 conv3_block1_1_bn (BatchNormal  (None, 64, 64, 128)  512        ['conv3_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block1_1_relu (Activatio  (None, 64, 64, 128)  0          ['conv3_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block1_2_conv (Conv2D)   (None, 64, 64, 32)   36864       ['conv3_block1_1_relu[0][0]']    
                                                                                                  
 conv3_blo

                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 64, 64, 32)   36864       ['conv3_block5_1_relu[0][0]']    
                                                                                                  
 conv3_block5_concat (Concatena  (None, 64, 64, 288)  0          ['conv3_block4_concat[0][0]',    
 te)                                                              'conv3_block5_2_conv[0][0]']    
                                                                                                  
 conv3_block6_0_bn (BatchNormal  (None, 64, 64, 288)  1152       ['conv3_block5_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block6_0_relu (Activatio  (None, 64, 64, 288)  0          ['conv3_block6_0_bn[0][0]']      
 n)       

 lization)                                                                                        
                                                                                                  
 conv3_block10_0_relu (Activati  (None, 64, 64, 416)  0          ['conv3_block10_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block10_1_conv (Conv2D)  (None, 64, 64, 128)  53248       ['conv3_block10_0_relu[0][0]']   
                                                                                                  
 conv3_block10_1_bn (BatchNorma  (None, 64, 64, 128)  512        ['conv3_block10_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_blo

...

                                                                                                  
 multiply_3 (Multiply)          (None, 64, 64, 256)  0           ['activation_43[0][0]',          
                                                                  'conv2d_45[0][0]']              
                                                                                                  
 add_5 (Add)                    (None, 64, 64, 256)  0           ['multiply_3[0][0]',             
                                                                  'conv2d_45[0][0]']              
                                                                                                  
 batch_normalization_32 (BatchN  (None, 64, 64, 256)  1024       ['add_5[0][0]']                  
 ormalization)                                                                                    
                                                                                                  
 activatio

                                                                 ]']                              
                                                                                                  
 activation_52 (Activation)     (None, 1, 1, 128)    0           ['conv2d_56[0][0]']              
                                                                                                  
 conv2d_57 (Conv2D)             (None, 1, 1, 128)    16512       ['activation_52[0][0]']          
                                                                                                  
 activation_53 (Activation)     (None, 1, 1, 128)    0           ['conv2d_57[0][0]']              
                                                                                                  
 multiply_5 (Multiply)          (None, 128, 128, 12  0           ['activation_53[0][0]',          
                                8)                                'conv2d_55[0][0]']              
          

                                                                                                  
 activation_61 (Activation)     (None, 256, 256, 64  0           ['conv2d_64[0][0]']              
                                )                                                                 
                                                                                                  
 multiply_6 (Multiply)          (None, 256, 256, 64  0           ['activation_61[0][0]',          
                                )                                 'max_pooling2d[0][0]']          
                                                                                                  
 concatenate_19 (Concatenate)   (None, 256, 256, 12  0           ['activation_59[0][0]',          
                                8)                                'multiply_6[0][0]']             
                                                                                                  
 conv2d_65

 conv2d_transpose_8 (Conv2DTran  (None, 512, 512, 32  4128       ['activation_68[0][0]']          
 spose)                         )                                                                 
                                                                                                  
 batch_normalization_49 (BatchN  (None, 512, 512, 32  128        ['conv2d_transpose_8[0][0]']     
 ormalization)                  )                                                                 
                                                                                                  
 activation_69 (Activation)     (None, 512, 512, 32  0           ['batch_normalization_49[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_73 (Conv2D)             (None, 512, 512, 32  1056        ['activation_69[0][0]']          
          

                                                                                                  
 activation_77 (Activation)     (None, 512, 512, 32  0           ['batch_normalization_53[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_82 (Conv2D)             (None, 512, 512, 32  9248        ['activation_77[0][0]']          
                                )                                                                 
                                                                                                  
 concatenate_24 (Concatenate)   (None, 512, 512, 96  0           ['add_8[0][0]',                  
                                )                                 'conv2d_82[0][0]',              
                                                                  'conv2d_80[0][0]']              
          